Image captioning is the task of predicting a caption for a given image. Common real world applications of it include aiding visually impaired people that can help them navigate through different situations. Therefore, image captioning helps to improve content accessibility for people by describing images to them.

This guide shows how to:
1. Fine-tune an image captioning model.
2. Use the fine-tuned model for inference.

# Libraries

In [ ]:
pip install transformers datasets evaluate -q
pip install jiwer -q